In [1]:
from pandas import DataFrame
import numpy as np
from math import log2
import random

# Вариант 1 (Деревья). Нерабочий

In [2]:
trainPath = "../data/train.data.cvs"
rowData = DataFrame.from_csv(trainPath, index_col=False).as_matrix()

In [3]:
def relNormalize(rel, max_rel=20):
    norm_rel = rel - min(rel)
    if max(norm_rel) != 0:
        norm_rel = norm_rel * max_rel / max(norm_rel)
    return norm_rel

In [2]:
class QueryData:
    def __init__(self, X, rel, T_NDCG=5):
        self._X = np.copy(X)
        self._n = self._X.shape[0]
        self._rel = np.copy(rel)
        self._T_NDCG = T_NDCG
        self._predcomputeNDCG()
        
    def _predcomputeNDCG(self):        
        right_order = np.argsort(self._rel)[::-1]
        self._X = self._X[right_order]    
        self._rel = self._rel[right_order]
        self._2_rel = 2 ** self._rel - 1   
        log2_order = np.log2(np.array(range(self._n)) + 2)
        self._max_DCG = sum(self._2_rel[:self._T_NDCG] / log2_order[:self._T_NDCG])
   
    def getX(self):
        return self._X
    
    def getY(self):
        return self._rel
        
    def getSwapNDCGMatrix(self, rel):
        order = np.array(np.argsort(rel)[::-1]) + 1
        log2_order = np.log2(order + 1)
        elem_DCG = self._2_rel / log2_order
        matr_elem_DCG = np.tile(elem_DCG, (self._n, 1))
        matr_swap_elem_DCG = self._2_rel.reshape((self._n, 1)) / log2_order.reshape((1, self._n)) 
        
        DCG = sum(elem_DCG[order <= self._T_NDCG])        
        lambda_mtr = np.full((self._n, self._n), DCG)
        lambda_mtr = lambda_mtr - matr_elem_DCG - matr_elem_DCG.T + matr_swap_elem_DCG + matr_swap_elem_DCG.T
        no_null_swap = ((order <= self._T_NDCG).reshape((self._n, 1)) + (order <= self._T_NDCG).reshape((1, self._n))) > 0 
        lambda_mtr = lambda_mtr * no_null_swap
        if self._max_DCG != 0:
            return lambda_mtr / self._max_DCG
        else:
            return lambda_mtr
    
    def getNDCG(self, rel):
        order = np.array(np.argsort(rel)[::-1]) + 1
        log2_order = np.log2(order + 1)
        elem_DCG = self._2_rel / log2_order
        DCG = sum(elem_DCG[order <= self._T_NDCG])
        if self._max_DCG != 0:
            return DCG / self._max_DCG
        else:
            return DCG

In [8]:
class QueryData:
    def __init__(self, X, rel):
        uniq_rel = np.unique(rel)
        self._X = np.empty((uniq_rel.shape[0], X.shape[1]))
        self._rel = np.empty(uniq_rel.shape[0])
        for i, val in enumerate(uniq_rel):
            self._X[i] = (X[rel == val])[0]
            self._rel[i] = (rel[rel == val])[0]
            
        #self._X = np.copy(X)
        self._n = self._X.shape[0]
        #self._rel = np.copy(rel)
        
        self._right_order = np.argsort(self._rel)[::-1]
        self._2_rel = 2 ** self._rel[self._right_order] - 1   
        self._log2_order = np.log2(np.array(range(self._n)) + 2)
        
    def maxDCG(self, T_NDCG=5):        
        return sum(self._2_rel[:T_NDCG] / self._log2_order[:T_NDCG])
   
    def getX(self):
        return self._X
    
    def getY(self):
        return self._rel
        
    def getSwapNDCGMatrix(self, rel, T_NDCG):
        rel = rel[self._right_order]
        order = np.array(np.argsort(rel)[::-1]) + 1
        log2_order = np.log2(order + 1)
        elem_DCG = self._2_rel / log2_order
        matr_elem_DCG = np.tile(elem_DCG, (self._n, 1))
        matr_swap_elem_DCG = self._2_rel.reshape((self._n, 1)) / log2_order.reshape((1, self._n)) 
        
        DCG = sum(elem_DCG[order <= T_NDCG])        
        lambda_mtr = np.full((self._n, self._n), DCG)
        lambda_mtr = lambda_mtr - matr_elem_DCG - matr_elem_DCG.T + matr_swap_elem_DCG + matr_swap_elem_DCG.T
        no_null_swap = ((order <= T_NDCG).reshape((self._n, 1)) + (order <= T_NDCG).reshape((1, self._n))) > 0 
        lambda_mtr = lambda_mtr * no_null_swap
        max_DCG = self.maxDCG(T_NDCG)
        if max_DCG != 0:
            return lambda_mtr / max_DCG 
        else:
            return lambda_mtr
    
    def getNDCG(self, rel, T_NDCG):
        rel = rel[self._right_order]
        order = np.array(np.argsort(rel)[::-1]) + 1
        log2_order = np.log2(order + 1)
        elem_DCG = self._2_rel / log2_order
        DCG = sum(elem_DCG[order <= T_NDCG])
        max_DCG = self.maxDCG(T_NDCG)
        if max_DCG  != 0:
            return DCG / max_DCG
        else:
            return DCG

In [9]:
queries = rowData[:, -1]
uniq_queries = np.unique(queries)
queries_train_data = []
for q in uniq_queries:
    xy = rowData[queries == q][:, :-1]
    queries_train_data.append(QueryData(xy[:, 1:], relNormalize(xy[:, 1])))

Save 1/5 train dataset

In [3]:
queries = rowData[:, -1]
uniq_queries = np.unique(queries)
random.shuffle(uniq_queries)
small_queries = uniq_queries[: uniq_queries.shape[0] / 5]
filter_for_queries = np.sum((queries == x for x in small_queries)) > 0
rowData2 = rowData[filter_for_queries]
np.save("small_train", rowData2)

/home/nina/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [1]:
from sklearn.tree import DecisionTreeRegressor as DT

class LambdaRankTrees :
    def __init__ (self, learning_rate= 1, n_estimators=100, sigma=1, start_depth=6) :
        self._learning_rate = learning_rate
        self._n_estimators = n_estimators
        self._sigma = sigma
        self._start_depth = start_depth
        self._trees = None
        
    def _getTrainX(self, queries_data):
        n = 0
        m = queries_data[0].getX().shape[1]
        for query_data in queries_data:
            n += query_data._n
            
        X = np.empty((n, m), dtype=np.float64)
        indexs = []
        cur_index = 0
        for query_data in queries_data:
            cur_n = query_data._n
            X[cur_index:cur_index + cur_n] = query_data.getX()
            indexs.append(range(cur_index,cur_index + cur_n))
            cur_index += cur_n            
        return X, indexs
    
    def _getGradient(self, queries_data, h, indexs_data, T_NDCG):
        g = np.empty(h.shape[0], dtype=np.float64)
        for i, indexs in enumerate(indexs_data):
            query_data = queries_data[i]
            rel = h[indexs]
            rel_n = rel.shape[0]
            lambda_matr = -self._sigma / (1 + np.exp(rel.reshape((rel_n, 1)) - rel.reshape((1, rel_n)))) * query_data.getSwapNDCGMatrix(rel, T_NDCG)
            tril = np.tril(lambda_matr, k=-1)
            lambda_vector =  np.sum(tril, axis=0) - np.sum(tril.T, axis=0)
            g[indexs] = lambda_vector
        return g
    
    def _getNDCG(self, queries_data, h, indexs_data, T_NDCG):
        ndcg = 0
        for i, indexs in enumerate(indexs_data):
            query_data = queries_data[i]
            rel = h[indexs]
            ndcg+= query_data.getNDCG(rel, T_NDCG)
        return ndcg / len(indexs_data)
    
    def fit(self, queries_data, max_add_iteration=10, persent_valid=0.4) :
        random.shuffle(queries_data)        
        count_valid = int(persent_valid * len(queries_data))
        data_valid = queries_data[:count_valid]
        data_train = queries_data[count_valid:]
        
        X_train, index_train = self._getTrainX(data_train)
        X_valid, index_valid = self._getTrainX(data_valid)
        
        self._trees = []
        #d_tree = DT(max_depth=self._start_depth)
        #d_tree.fit(X_train, np.zeros(X_train.shape[0]))        
        #self._trees.append(d_tree)
        h_train = np.zeros(X_train.shape[0]) #d_tree.predict(X_train) * self._learning_rate   
        h_valid = np.zeros(X_valid.shape[0])# d_tree.predict(X_valid) * self._learning_rate 
        
        for iteration in range(self._n_estimators - 1) :
            g = self._getGradient(data_train, h_train, index_train, 200)
            norm_g = np.linalg.norm(g)            
            """
            if norm_g > 3 :
                d_tree = dt.DecisionTree('mse', self.max_depth, collect_gains_features=self.collect_gains_features)
            else :
                d_tree = dt.DecisionTree('mse', 3 * self.max_depth, collect_gains_features=self.collect_gains_features)
            """
            #data = np.hstack((x, g.reshape(N, 1)))
            d_tree = DT(max_depth=self._start_depth)
            d_tree.fit(X_train, -g)
            self._trees.append(d_tree)
            h_train += self._learning_rate * d_tree.predict(X_train)
            h_valid += self._learning_rate * d_tree.predict(X_valid)
            
            print(iteration, self._getNDCG(data_train, h_train, index_train, 200), 
                  self._getNDCG(data_valid, h_valid, index_valid, 200),
                  self._getNDCG(data_train, h_train, index_train, 5),
                  self._getNDCG(data_valid, h_valid, index_valid, 5),
                  min(g), max(g), norm_g)


In [11]:
lambdaRank = LambdaRankTrees()
lambdaRank.fit(queries_train_data)

0 0.497820015453 0.493057026561 0.486061325237 0.479736631324 -2.14857107211 1.82708231185 73.7833373181
1 0.497820007461 0.493073221956 0.486061317244 0.479736613523 -3.46779615142 2.88473461137 116.339052056
2 0.497820007461 0.493073239755 0.486061317244 0.479736631324 -4.02074875303 3.34028318043 139.598143368
3 0.497819330594 0.493072423385 0.486060640377 0.47973577421 -4.11171509198 3.44589787724 145.437299392
4 0.497819330594 0.493072642521 0.486060640377 0.47973577421 -4.12811386045 3.47669901588 146.849519973
5 0.497819330594 0.49305579834 0.486060640377 0.479735245818 -4.13180070864 3.48749962038 147.25210405
6 0.497819330594 0.493056326731 0.486060640377 0.47973577421 -4.13273868287 3.49155727026 147.385395035
7 0.497819330594 0.493056326731 0.486060640377 0.47973577421 -4.13299198138 3.49312705513 147.435256997
8 0.497819330594 0.493056326731 0.486060640377 0.47973577421 -4.13306225311 3.49374410563 147.456091755
9 0.497819330594 0.493056144375 0.486060640377 0.47973577421 -

KeyboardInterrupt: 

In [ ]:
    """

    def predict(self, x) :
        y = np.zeros(x.shape[0])
        for tree in self.trees:
            y = y + self.learning_rate * tree.predict(x)

        return np.apply_along_axis(lambda x: (1.0 / (1 + np.exp(-x)) > 0.5) * 1.0, 0, y)

    def score(self, x, y) :
        h = np.zeros(x.shape[0])
        score_y = []
        for tree in self.trees:
            h = h + self.learning_rate * tree.predict(x)
            y_pred = np.apply_along_axis(lambda x: (1.0 / (1 + np.exp(-x)) > 0.5) * 1.0, 0, h)
            score_y.append(accuracy_score(y, y_pred))
        return np.array(score_y)
        
    def f1_score(self, x, y) :
        h = np.zeros(x.shape[0])
        score_y = []
        for tree in self.trees:
            h = h + self.learning_rate * tree.predict(x)
            y_pred = np.apply_along_axis(lambda x: (1.0 / (1 + np.exp(-x)) > 0.5) * 1.0, 0, h)
            score_y.append(f1_score(y, y_pred))
        return np.array(score_y)
        
    def f1_macro(self, x, y) :
        h = np.zeros(x.shape[0])
        score_y = []
        score_y1 = []
        score_y2 = []
        for tree in self.trees:
            h = h + self.learning_rate * tree.predict(x)
            y_pred = np.apply_along_axis(lambda x: (1.0 / (1 + np.exp(-x)) > 0.5) * 1.0, 0, h)
            labels = unique_labels(y, y_pred)
            f1 = f1_score(y, y_pred, labels=labels, average=None)
            f1_macro = 0
            for i, label in enumerate(labels):
                f1_macro += f1[i]    
            score_y1.append(f1[0])
            score_y2.append(f1[1])
            score_y.append(f1_macro)
        return np.array(score_y)/ 2.0, np.array(score_y1), np.array(score_y2)
    """

# Вариант 2 с нейросетями (недоделанный)

In [3]:
import random
from operator import add, mul, sub
from math import exp, ceil, log
import numpy as np
from skimage.io import imread
from skimage.color import rgb2grey
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib import gridspec


In [5]:
class LambdaRankEstimator:
    def __init__(self) :
        pass
    def init(self, count_input, count_layers=1, act_funcs=None, der_act_func=None,
             count_neurals_layer=None, learning_rate=0.1, shuffle = True) :
        defalt_count_neural = 10
        if count_neurals_layer is None:
            count_neurals_layer = []
            for l in range(count_layers):
                count_neurals_layer.append(defalt_count_neural)
        if act_funcs is not None :
            self.act_funcs = act_funcs
            self.der_act_func = der_act_func
        else :   
            self.act_funcs = []
            self.der_act_func = []
            for l in range(count_layers):
                self.act_funcs.append(logistic_activation_1) # max_0)
                self.der_act_func.append(der_logistic_activation_1) # der_max_0
        self.learning_rate = learning_rate
        self.count_output = 1
        self.shuffle = shuffle
        count_neurals_layer.append(self.count_output)
        self.act_funcs.append(equal_func)
        self.der_act_func.append(equal_func)
        #self.function_error = mean_square
        #self.dE_dz_last = der_mean_square - надо заменить!!!!!!!!!!!!!!!
        self.create_network(count_input, count_layers + 1, count_neurals_layer)
        self.initialize()

    def fit(self, data, add_step = 3, add_iteration=100, max_epoche=3000, sigma=1,
            coeff_R1=0, coeff_R2=0) :
        self.coeff_R1 = coeff_R1
        self.begin_coeff_R1 = coeff_R1
        self.coeff_R2 = coeff_R2
        self.begin_coeff_R2 = coeff_R2
        self.validation_error = []

        if self.shuffle:
            random.shuffle(data)
        
        count_valid = int(4 * len(data) / 10)
        data_valid = data[:count_valid]
        data_train = data[count_valid:]

        best_error = 1e100
        best_weight = None
        number_epoche = None
        cur_step = 0
        current_epoche = 0
        while True :
            #epoche
            random.shuffle(data_train)
            epoche_error = 0
            for iteration in range(len(data_train)):#range(self.batch_count) :
                x = data_train[iteration][:, :-1]
                y_true = data_train[iteration][:, -1]
                y_pred = self.forward_propogation(x)
                print(y_pred.shape, y_true.shape)
                break
            break

            """
            self.init_add_weight()
            error = 0
            for j in range(size_batch) :
                n = iteration * size_batch + j
                answer = self.forward_propogation(data_train[n])
                self.back_propogation(data_train[n], answer, y_train[n])
                error = error + self.function_error(y_train[n], answer)
            self.add_mean_gradient(size_batch)
            epoche_error = epoche_error + error
            """
            """
            epoche_error = epoche_error/ (self.batch_count * size_batch)
            print "Epoche error", epoche_error, 
            
            error = 0
            answer = self._predict_(data_x_valid)
            for i, y in enumerate(answer):
                error = error + self.function_error(y_valid[i], y)
            error = error / answer.shape[0]            
            print "Validation error", error
            
            if error <= best_error:
              #  print "New best error", error
                best_error = error
                best_weight = self.get_copy_weight(self.weight_network)
                self.validation_error.append(error)
                number_epoche = current_epoche
                
            
            if  number_epoche + add_iteration <= current_epoche:# and cur_step < add_step:
                cur_step = cur_step + 1                
                self.weight_network = self.get_copy_weight(best_weight)
                self.learning_rate = self.learning_rate / 2.0
                number_epoche = current_epoche
                print "New learning rate", self.learning_rate
                if cur_step > add_step:
                    self.count_epoche_for_learn = current_epoche - 1
                    break
            
            self.coeff_R1 = self.begin_coeff_R1 / ((current_epoche) ** 3 + 1)
            self.coeff_R2 = self.begin_coeff_R2 / ((current_epoche) ** 3 + 1)
            for i, x in enumerate(self.KL_probability):
                if x is not None:
                    prob, coeff = self.KL_begin_probability[i]
                    self.KL_probability[i] = (prob, coeff / ((current_epoche) ** 3 + 1))

            current_epoche = current_epoche + 1    
            if current_epoche > max_epoche:
                self.weight_network = self.get_copy_weight(best_weight)
                self.count_epoche_for_learn = current_epoche - 1
                break
            """


        
    def get_copy_weight(self, weights) :
        new_weight = []
        for w in weights:
            new_weight.append(w.copy())
        return new_weight
        
    def _predict_(self, data) :
        y = np.empty((data.shape[0], self.count_output))
        for i, x in enumerate(data):
            y[i, :] = self.forward_propogation(x)
        return y
        
    def predict(self, data):
        y_answer = self._predict_(data)
        if self.isClassification:
            y_pred = np.empty(y_answer.shape[0])
            for i, y in enumerate(y_answer):            
                y_pred[i] = self.uniq_label[y.argmax()]
            return y_pred        
        return y_answer
    
    def init_add_weight(self) :
        self.add_weight = []
        for layer in self.weight_network:
            self.add_weight.append(np.zeros(layer.shape))

    def create_network(self, count_input, count_layers, count_neurals_layer) :
        self.weight_network = []
        self.KL_probability = []
        self.KL_begin_probability = []
        count_neurals_layer = count_neurals_layer[:]
        count_neurals_layer.insert(0, count_input)    
        for l in range(1, count_layers + 1) :
            self.weight_network.append(np.zeros((count_neurals_layer[l], count_neurals_layer[l-1])))
            self.KL_probability.append(None)
            self.KL_begin_probability.append(None)

    def initialize(self, mean=0, var=1.0/800) :
        for l, layer in enumerate(self.weight_network) :
            var = 1.0 / layer.shape[1] #/ 10.0 #* 25
            weights = [[random.gauss(mean, var) for i in range(layer.shape[1])] for j in range(layer.shape[0])]
            self.weight_network[l] = np.array(weights)

    def set_KL_probality(self, layer, prob, coeff):
        self.KL_probability[layer] = (prob, coeff)
        self.KL_begin_probability[layer] = (prob, coeff)
        
    def forward_propogation(self, data) :
        cur_x = data.T
        self.x = []
        self.dy_dz = []
        self.x.append(cur_x)
        for l, weight_matrix in enumerate(self.weight_network):
            z = np.dot(weight_matrix, cur_x)
            func = (self.act_funcs)[l]
            der_func = self.der_act_func[l]
            cur_x = func(z)
            self.x.append(cur_x)
            self.dy_dz.append(der_func(z))
        return cur_x
        
    def back_propogation(self, data, answer, true_y) :
        dE_dz = self.dE_dz_last(true_y, answer)        
        l = len(self.add_weight)
        prev_x = self.x[l - 1]
        dE_dz = dE_dz.reshape((dE_dz.shape[0], 1))
        prev_x = prev_x.reshape((1, prev_x.shape[0]))
        self.add_weight[l - 1] = self.add_weight[l - 1]  + np.dot(dE_dz, prev_x
                ) + self.coeff_R1 * sign(self.weight_network[l - 1]
                ) + self.coeff_R2 * self.weight_network[l - 1]
        for i in range(l - 2, -1, -1):
            tmp1_matrix = np.dot(self.weight_network[i+1].T , dE_dz)
            prev_x = self.x[i]
            prev_x = prev_x.reshape((1, prev_x.shape[0]))
            dy_dz = self.dy_dz[i]
            dy_dz = dy_dz.reshape((dy_dz.shape[0], 1))
            tmp2_matrix = np.dot(dy_dz, prev_x)
            add_weight = tmp2_matrix * tmp1_matrix  
            add_R1 = self.coeff_R1 * sign(self.weight_network[i]) 
            add_R2 = self.coeff_R2 * self.weight_network[i]         
            if self.KL_probability[i] is not None:
                prob, coeff = self.KL_probability[i]
                add_dE_dz = - dy_dz * (log(prob) / self.weight_network[i].shape[0])
                add_KL_reg = np.dot(add_dE_dz, prev_x) * coeff
            else:
                add_dE_dz = 0
                add_KL_reg = 0
            add_weight = add_weight + add_R1 + add_R2 + add_KL_reg       
            
            self.add_weight[i] = self.add_weight[i]  + add_weight
            dE_dz = tmp1_matrix * dy_dz + add_dE_dz
            
    def add_mean_gradient(self, count_iteration):
        for i, layer in enumerate(self.add_weight) : 
           # print "layer" , i
           # print "w", self.weight_network[i][:3, :3]
           # print "add", (- self.learning_rate * layer / count_iteration)[:3, :3]
            self.weight_network[i] = self.weight_network[i] - self.learning_rate * layer / count_iteration
      
    def print_weight(self):
        for l, layer in enumerate(self.weight_network) :   
            print("LAYER:", l, "shape", layer.shape)
            for n, neuron in enumerate(layer):
                print("neuron:", n)
                for weight in neuron:
                    print(weight)
                print('')

In [34]:
estimator = LambdaRankEstimator()
estimator.init(data[0].shape[1] - 1)

In [35]:
estimator.fit(data)

(1, 109) (109,)


In [4]:
def sign(data):
    return (data > 0) * 1.0  - (data < 0) * 1.0

def mean_square(y, answer):
    return sum((y - answer) ** 2) / 2.0
def der_mean_square(y, answer):
    return answer - y
    
def entropy(y, answer) :
    return sum(-y * np.log(answer + 1e-20))
def der_soft_max(y, answer):
    return answer - y
    
def equal_func(z) :
    return z  
def one_func (z) :
    return 1.0;
    
def logistic_activation_1(z) :
    return logistic_activation_a(1.0, z)
def der_logistic_activation_1(z) :
    return der_logistic_activation_a(1.0, z)
    
def max_0(z) :
    return (z > 0) * z
def der_max_0(z):
    return (z > 0) * 1.0

                
def logistic_activation_a(a, z) :
    return 1.0 / (1 + np.exp(-a * z))
def der_logistic_activation_a(a, z):
    return a*logistic_activation_a(a, z)*(1.0 - logistic_activation_a(a, z))
    
def load_data() :
    path = './big_alphabet_29x29/mutant-'
    count_char = 25
    count_example = 8
    image = rgb2grey(imread('./big_alphabet_29x29/mutant-0-0-0.bmp'))
    size = image.shape[0] * image.shape[1]
    data_x = np.zeros((count_char * count_example, size))
    y = np.zeros(count_char * count_example)    

        
    for char in range(count_char) :
        for i in range(count_example):
            path_img = path + str(char) + '-' + str(i) + '-0.bmp'
            data_x[char * count_example + i, :] = rgb2grey(imread(path_img)).reshape(size)
            y[char * count_example + i] = char
    data_x =  data_x - 0.5
  #  data_x = data_x / np.max(np.abs(data_x))
    return data_x, y 
    
def acurancy(y_pred, y_true):
    return float(sum(y_pred == y_true)) / y_pred.shape[0]

# Вариант 3 (Деревья). Более-менее

In [1]:
from pandas import DataFrame
import numpy as np
from math import log2
import random
from sklearn.tree import DecisionTreeRegressor as DT

In [2]:
def relNormalize1(rel, max_rel=19):
    norm_rel = rel - min(rel)
    if max(norm_rel) != 0:
        norm_rel = norm_rel * max_rel / max(norm_rel) + 1
    return norm_rel

def relNormalize2(rel):
    uniq_rel = np.unique(rel)
    uniq_rel = sorted(uniq_rel)
    norm_rel = np.empty(rel.shape)
    for i, val in enumerate(uniq_rel):
        norm_rel[rel==val] = i + 1
    return norm_rel

In [3]:
class QueryData:
    def __init__(self, X, rel):
        self._X = np.copy(X)
        self._n = self._X.shape[0]
        self._rel = np.copy(rel)

        right_order = np.argsort(self._rel)[::-1]
        self._X = self._X[right_order]
        self._rel = self._rel[right_order]
        self._2_rel = 2 ** self._rel - 1
        self._log2_order = np.log2(np.array(range(self._n)) + 2)
        
    def maxDCG(self, T_NDCG):
        return sum(self._2_rel[:T_NDCG] / self._log2_order[:T_NDCG])
   
    def getX(self):
        return self._X
    
    def getY(self):
        return self._rel
        
    def getSwapNDCGMatrix(self, rel, T_NDCG):
        order = np.empty(rel.shape)
        sort_rel = sorted(zip(rel, range(rel.shape[0])), key=lambda x: x[0], reverse=True)
        order[list(map(lambda x: x[1], sort_rel))] = range(1, rel.shape[0] + 1)
        
        log2_order = np.log2(order + 1)
        elem_DCG = self._2_rel / log2_order
        matr_elem_DCG = np.tile(elem_DCG, (self._n, 1))
        matr_swap_elem_DCG = self._2_rel.reshape((self._n, 1)) / log2_order.reshape((1, self._n)) 
        
        lambda_mtr = - matr_elem_DCG - matr_elem_DCG.T + matr_swap_elem_DCG + matr_swap_elem_DCG.T
        no_null_swap = ((order <= T_NDCG).reshape((self._n, 1)) + (order <= T_NDCG).reshape((1, self._n))) > 0 
        lambda_mtr = np.abs(lambda_mtr * no_null_swap)
        max_DCG = self.maxDCG(T_NDCG)
        if max_DCG != 0:
            return lambda_mtr / max_DCG 
        else:
            return lambda_mtr
    
    def getNDCG(self, rel, T_NDCG):
        order = np.empty(rel.shape)
        sort_rel = sorted(zip(rel, range(rel.shape[0])), key=lambda x: x[0], reverse=True)
        order[list(map(lambda x: x[1], sort_rel))] = range(1, rel.shape[0] + 1)
        log2_order = np.log2(order + 1)
        elem_DCG = self._2_rel / log2_order
        DCG = sum(elem_DCG[order <= T_NDCG])
        max_DCG = self.maxDCG(T_NDCG)
        if max_DCG  != 0:
            return DCG / max_DCG
        else:
            return DCG
        
    def getCountErrorPair(self, rel):
        _n = rel.shape[0]
        order = np.argsort(rel)[::-1]
        pairs = ((- order.reshape((_n, 1)) + order.reshape((1, _n))) > 0)
        no_null = ((self._rel.reshape((_n, 1)) - self._rel.reshape((1, _n))) != 0)
        count = sum(sum((np.tril(pairs * no_null))))
        return count

In [39]:
rel = np.array([5, 5, 4, 4, 3, 2, 1])
X = np.array([1, 2, 3, 4, 5, 1, 1])
query_data = QueryData(X, rel)
query_data.getCountErrorPair(np.array([4, 5, 5, 4, 3, 2, 1]))

[2 1 3 0 4 5 6]
[[False False  True False  True  True  True]
 [ True False  True False  True  True  True]
 [False False False False  True  True  True]
 [ True  True  True False  True  True  True]
 [False False False False False  True  True]
 [False False False False False False  True]
 [False False False False False False False]]
[[False False  True  True  True  True  True]
 [False False  True  True  True  True  True]
 [ True  True False False  True  True  True]
 [ True  True False False  True  True  True]
 [ True  True  True  True False  True  True]
 [ True  True  True  True  True False  True]
 [ True  True  True  True  True  True False]]


2

In [24]:
class LambdaRankTrees :
    def __init__ (self, learning_rate=0.5, n_estimators=1000, sigma=1, start_depth=5) :
        self._learning_rate = learning_rate
        self._n_estimators = n_estimators
        self._sigma = sigma
        self._start_depth = start_depth
        self._trees = None
        
    def _getTrainX(self, queries_data):
        n = 0
        m = queries_data[0].getX().shape[1]
        for query_data in queries_data:
            n += query_data._n
            
        X = np.empty((n, m), dtype=np.float64)
        Y = np.empty(n, dtype=np.float64)
        indexs = []
        cur_index = 0
        for query_data in queries_data:
            cur_n = query_data._n
            X[cur_index:cur_index + cur_n] = query_data.getX()
            Y[cur_index:cur_index + cur_n] = query_data.getY()
            indexs.append(range(cur_index,cur_index + cur_n))
            cur_index += cur_n            
        return X, Y, indexs
    
    def _getGradient(self, queries_data, h, indexs_data, T_NDCG):
        g = np.empty(h.shape[0], dtype=np.float64)
        for i, indexs in enumerate(indexs_data):
            query_data = queries_data[i]
            rel = h[indexs]
            rel_n = rel.shape[0]
            lambda_matr = np.full((rel_n, rel_n), 1, dtype=np.int) #(- self._sigma / (1 + np.exp(self._sigma * (rel.reshape((1, rel_n)) - -rel.reshape((rel_n, 1)))))) #* query_data.getSwapNDCGMatrix(rel, T_NDCG)

            filter_matr = (query_data._rel.reshape((1, rel_n)) - query_data._rel.reshape((rel_n, 1))) != 0
            lambda_matr *= filter_matr
            tril = np.tril(lambda_matr)
            lambda_vector =  np.sum(tril, axis=0) - np.sum(tril.T, axis=0)
            g[indexs] = lambda_vector
        return g
    
    def _getNDCG(self, queries_data, h, indexs_data, T_NDCG):
        ndcg = 0
        for i, indexs in enumerate(indexs_data):
            query_data = queries_data[i]
            rel = h[indexs]
            ndcg+= query_data.getNDCG(rel, T_NDCG)
        return ndcg / len(indexs_data)
    
    def _countErrorPair(self, queries_data, h, indexs_data):
        count = 0
        for i, indexs in enumerate(indexs_data):
            query_data = queries_data[i]
            rel = h[indexs]
            count += query_data.getCountErrorPair(rel)
        return count        
    
    def fit(self, queries_data, persent_valid=0.2, persent_train=0.1) :
        random.seed(1234)
        random.shuffle(queries_data)        
        count_valid = int(persent_valid * len(queries_data))
        data_valid = queries_data[:count_valid]
        data_train = queries_data[count_valid:]
        
        X_train, y_train, index_train = self._getTrainX(data_train)
        X_valid, y_valid, index_valid = self._getTrainX(data_valid)
        #count_train = int(X_train.shape[0] * persent_train)
        #range_train = list(range(0, X_train.shape[0]))
        
        self._trees = []
        self._score_train = []
        self._score_valid = []

        #d_tree = DT(max_depth=self._start_depth * 10)        
        #y_start_train = np.array([random.gauss(mu=1, sigma=1) for i in range(count_train)])
        #d_tree.fit(X_train[:count_train], y_start_train) #np.zeros(X_train.shape[0]))
        #self._trees.append(d_tree)

        #h_train = d_tree.predict(X_train) # np.full(X_train.shape[0], 0) #d_tree.predict(X_train) * self._learning_rate
        #h_valid = d_tree.predict(X_valid) # np.full(X_valid.shape[0], 0)# d_tree.predict(X_valid) * self._learning_rate

        h_train = np.zeros(X_train.shape[0])
        h_valid = np.zeros(X_valid.shape[0])
        for iteration in range(self._n_estimators) :
            print(iteration, self._countErrorPair(data_train, h_train, index_train),
                  self._countErrorPair(data_valid, h_valid, index_valid))
            
            g = self._getGradient(data_train, h_train, index_train, 10)
            norm_g = np.linalg.norm(g)            
            """
            if norm_g > 3 :
                d_tree = dt.DecisionTree('mse', self.max_depth, collect_gains_features=self.collect_gains_features)
            else :
                d_tree = dt.DecisionTree('mse', 3 * self.max_depth, collect_gains_features=self.collect_gains_features)
            """
            #random.shuffle(range_train)
            d_tree = DT(max_depth=self._start_depth)
            d_tree.fit(X_train, g)
            #d_tree.fit(X_train[range_train[:count_train]], -g[range_train[:count_train]])
            self._trees.append(d_tree)
            h_train += self._learning_rate * d_tree.predict(X_train)
            h_valid += self._learning_rate * d_tree.predict(X_valid)
            
           
              #self._getNDCG(data_train, h_train, index_train,10),
              #self._getNDCG(data_valid, h_valid, index_valid, 10),
              #self._getNDCG(data_train, h_train, index_train, 5),
              #self._getNDCG(data_valid, h_valid, index_valid, 5),
              #min(g), max(g), norm_g)
            #self._score_train.append(self._getNDCG(data_train, h_train, index_train, 5))
            #self._score_valid.append(self._getNDCG(data_valid, h_valid, index_valid, 5))
            
    def predict(self, X) :
        y = self._trees[0].predict(X)
        for tree in self._trees[1:]:
            y += self._learning_rate * tree.predict(X)
        return y


In [25]:
def loadData():
    trainPath = "../data/train.data.cvs"
    return DataFrame.from_csv(trainPath, index_col=False).as_matrix()

def loadSmallData():
    return np.load("small_train.npy")

In [7]:
rowData = loadData()
queries = rowData[:, -1]
uniq_queries = np.unique(queries)
queries_train_data = []
for q in uniq_queries:
    xy = rowData[queries == q][:, :-1]
    queries_train_data.append(QueryData(xy[:, 1:], relNormalize2(xy[:, 0])))

In [26]:
lambdaRank = LambdaRankTrees(learning_rate=0.01, n_estimators=100, sigma=1, start_depth=5)

In [27]:
lambdaRank.fit(queries_train_data[:100],  persent_valid=0.2, persent_train=1)

0 73529 21641
1 96453 31734
2 96453 31734
3 96453 31734
4 96453 31734
5 96453 31734
6 96453 31734
7 96453 31734
8 96453 31734
9 96453 31734
10 96453 31734
11 96453 31734
12 96453 31734
13 96453 31734
14 96453 31734
15 96453 31734
16 96453 31734
17 96453 31734
18 96453 31734


KeyboardInterrupt: 

In [17]:
from sklearn.externals import joblib
joblib.dump(lambdaRankank, "lambdaRank2.pkl")

['lambdaRank1.pkl']

In [3]:
from sklearn.externals import joblib
lambdaRank = joblib.load("lambdaRank1.pkl")

AttributeError: module '__main__' has no attribute 'LambdaRankTrees'

In [15]:
ans

array([ 1.03212926,  0.18316964,  0.87774511, ...,  1.45966731,
        0.69223217,  0.38776884])

In [55]:
testRow[:, -1]

array([  1.30000000e+01,   1.30000000e+01,   1.30000000e+01, ...,
         2.99980000e+04,   2.99980000e+04,   2.99980000e+04])

In [56]:
query_data = queries_train_data[10]
ans = lambdaRank.predict(query_data.getX())
print(query_data.getNDCG(ans, 5))

0.469031253098


In [32]:
print(ans.shape, query_data.getX().shape)

(207,) (207, 136)


In [62]:
rel = ans
order = np.empty(rel.shape)
sort_rel = sorted(zip(rel, range(rel.shape[0])), key=lambda x: x[0], reverse=True)
order[list(map(lambda x: x[1], sort_rel))] = range(1, rel.shape[0] + 1)
log2_order = np.log2(order + 1)
elem_DCG = query_data._2_rel / log2_order
DCG = sum(elem_DCG[order <= 5])

In [68]:
query_data.maxDCG(5)

2235622.7682765387

In [36]:
trees = lambdaRank._trees

In [37]:
len(trees)

24

In [5]:
def DCGScore(y_true, y_score, k=10, gains="exponential"):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    if gains == "exponential":
        gains = 2 ** y_true - 1
    elif gains == "linear":
        gains = y_true
    else:
        raise ValueError("Invalid gains option.")
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)

def NDCGScore(y_true, y_score, k=10, gains="exponential"):
    best = DCGScore(y_true, y_true, k, gains)
    actual = DCGScore(y_true, y_score, k, gains)
    return actual / best

def deltaNDCGScore(y_true, y_score, k=10, gains="exponential"):
    max_DCG = NDCGScore(y_true, y_true, k, gains)
    order = np.empty(y_score.shape)
    sort_y_score = sorted(zip(y_score, range(y_score.shape[0])), key=lambda x: x[0], reverse=True)
    order[list(map(lambda x: x[1], sort_y_score))] = range(1, y_score.shape[0] + 1)
    discounts = np.log2(order + 1)    
    if gains == "exponential":
        gains = 2 ** y_true - 1
    elif gains == "linear":
        gains = y_true
    else:
        raise ValueError("Invalid gains option.")        
    elem_DCG = gains / discounts
    _n = y_true.shape[0]
    matr_elem_DCG = np.tile(elem_DCG, (_n, 1))
    matr_swap_elem_DCG = gains.reshape((_n, 1)) / discounts.reshape((1, _n)) 

    lambda_mtr = - matr_elem_DCG - matr_elem_DCG.T + matr_swap_elem_DCG + matr_swap_elem_DCG.T
    no_null_swap = ((order <= k).reshape((_n, 1)) + (order <= k).reshape((1, _n))) > 0 
    lambda_mtr = np.abs(lambda_mtr * no_null_swap)
    
    if max_DCG != 0:
        return lambda_mtr / max_DCG 
    else:
        return lambda_mtr
    
def AverageNDCG(data, y_score, k):
    uniq_queries = np.unique(data[:, -1])
    ndcg = 0
    for q in uniq_queries:
        filt_q = (data[:, -1] == q)
        ndcg += NDCGScore(relNormalize2(data[filt_q, 1]), y_score[filt_q], k)
    return ndcg / uniq_queries.shape[0]

In [11]:
def relNormalize2(rel):
    uniq_rel = np.unique(rel)
    uniq_rel = sorted(uniq_rel)
    norm_rel = np.empty(rel.shape)
    for i, val in enumerate(uniq_rel):
        norm_rel[rel==val] = i + 1
    return norm_rel

In [6]:
y_true = np.array([7, 1, 2, 4])
y_score = np.array([1, 2, 3,4])
deltaNDCGScore(y_true, y_score, 10)

array([[  0.        ,   8.73475368,  24.83139624,  63.7642255 ],
       [  8.73475368,   0.        ,   0.26185951,   7.        ],
       [ 24.83139624,   0.26185951,   0.        ,   4.42884296],
       [ 63.7642255 ,   7.        ,   4.42884296,   0.        ]])

In [15]:
class LambdaRankForest:
    def __init__ (self, learning_rate, n_estimators, sigma, start_depth) :
        self._learning_rate = learning_rate
        self._n_estimators = n_estimators
        self._sigma = sigma
        self._start_depth = start_depth
        self._trees = []
        
        
    def fit(self, DATA, persent_valid, batch_size, T_NDCG):
        all_queries = DATA[:, -1]
        uniq_queries = np.unique(all_queries)
        random.shuffle(uniq_queries)        
        count_valid = int(persent_valid * uniq_queries.shape[0])
        
        valid_queries = uniq_queries[:count_valid]
        train_queries = uniq_queries[count_valid:]
        DATA_valid = np.concatenate([DATA[all_queries == q] for q in valid_queries], axis=0)
        DATA_train = np.concatenate([DATA[all_queries == q] for q in train_queries], axis=0)
        valid_queries = DATA_valid[:, -1]
        train_queries = DATA_train[:, -1]
        
        self._trees = []
        h_train = np.array([random.gauss(mu=0, sigma=1) for i in range(DATA_train.shape[0])])
        h_valid = np.zeros(DATA_valid.shape[0])
        
        iteration = 0
        while True:
            random.shuffle(train_queries)
            for i in range(int(train_queries.shape[0] / batch_size)):
                batch_queries = train_queries[i * batch_size: (i + 1) * batch_size]
                data = np.concatenate([DATA_train[train_queries == q] for q in batch_queries], axis=0)
                h_pred = np.concatenate([h_train[train_queries == q] for q in batch_queries])                
                X = data[:, 1:-1]
                Y_true = data[:, 0]                
                
                grad = np.empty(Y_true.shape)
                for q in batch_queries:
                    filter_query = (data[:, -1] == q)
                    h = h_pred[filter_query]
                    y = relNormalize2(Y_true[filter_query])
                    p = 1 + np.exp((h.reshape((h.shape[0], 1)) - h.reshape((1, h.shape[0]))) * self._sigma)
                    g = (- self._sigma / p) * deltaNDCGScore(y, h, T_NDCG)
                    g = np.tril(g)
                    
                    grad[filter_query] = np.sum(g, axis=0) - np.sum(g.T, axis=0)
                
                new_tree = DT(max_depth=self._start_depth)
                new_tree.fit(X, -grad)
                self._trees.append(new_tree)
                
                h_train += self._learning_rate * new_tree.predict(DATA_train[:, 1:-1])
                h_valid += self._learning_rate * new_tree.predict(DATA_valid[:, 1:-1])
                
                print (iteration,
                       AverageNDCG(DATA_train, h_train, 20),
                       AverageNDCG(DATA_valid, h_valid, 20), 
                       AverageNDCG(DATA_train, h_train, 5), 
                       AverageNDCG(DATA_valid, h_valid, 5), 
                       np.linalg.norm(-grad),
                       min(-grad),
                       max(-grad))
                iteration += 1
                if (iteration == self._n_estimators):
                    break
            if (iteration == self._n_estimators):
                break
    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        for tree in self._trees:
            y_pred += self._learning_rate * tree.predict(X)
        return y_pred

In [16]:
estimator = LambdaRankForest(learning_rate=1, n_estimators=100, sigma=1, start_depth=5)
estimator.fit(DATA=rowData[:int(rowData.shape[0]/ 2)], persent_valid=0.5, batch_size=100, T_NDCG=30)

0 0.268875510747 0.730170421273 0.127203439044 0.604668713833 inf -1.15945107722e+169 3.41909026892e+170
1 0.229356174581 0.827322343813 0.166895104059 0.854817754669 inf -9.0614516453e+215 2.79707958711e+217
2 0.198487081215 0.641814272362 0.132887473349 0.554431404046 inf -4.44312249886e+215 3.96709691678e+217


KeyboardInterrupt: 

In [9]:
def loadData():
    trainPath = "../data/train.data.cvs"
    return DataFrame.from_csv(trainPath, index_col=False).as_matrix()
rowData = loadData()

In [4]:
from pandas import DataFrame
import numpy as np
from math import log2
import random
from sklearn.tree import DecisionTreeRegressor as DT

In [5]:
result  = DataFrame.from_csv("result2", index_col=False).as_matrix()

In [7]:
result.shape

(241521, 2)

In [10]:
rels = result[:, 0]
queries = result[:, 1]

uniq_queries = np.unique(queries)
ans = np.empty((rels.shape[0], 2), dtype=np.int)
for q in uniq_queries:
    rel = rels[queries == q]    
    order = np.argsort(rel) + 1
    ans[queries == q, 0] = order 
    ans[queries == q, 1] = q
df = DataFrame(ans, columns=["DocumentId","QueryId"])
df.to_csv(open("result3", "w"), index=False)      

# Последний вариант

In [1]:
from pandas import DataFrame
import numpy as np
from math import log2
import random
from sklearn.tree import DecisionTreeRegressor as DT


def DCGScore(y_true, y_score, k=10, gains="exponential"):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    if gains == "exponential":
        gains = 2 ** y_true - 1
    elif gains == "linear":
        gains = y_true
    else:
        raise ValueError("Invalid gains option.")
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)

def NDCGScore(y_true, y_score, k=10, gains="exponential"):
    best = DCGScore(y_true, y_true, k, gains)
    actual = DCGScore(y_true, y_score, k, gains)
    return actual / best

def deltaNDCGScore(y_true, y_score, k=10, gains="exponential"):
    max_DCG = DCGScore(y_true, y_true, k, gains)
    order = np.empty(y_score.shape)
    sort_y_score = sorted(zip(y_score, range(y_score.shape[0])), key=lambda x: x[0], reverse=True)
    order[list(map(lambda x: x[1], sort_y_score))] = range(1, y_score.shape[0] + 1)
    discounts = np.log2(order + 1)
    if gains == "exponential":
        gains = 2 ** y_true - 1
    elif gains == "linear":
        gains = y_true
    else:
        raise ValueError("Invalid gains option.")
    elem_DCG = gains / discounts
    _n = y_true.shape[0]
    matr_elem_DCG = np.tile(elem_DCG, (_n, 1))
    matr_swap_elem_DCG = gains.reshape((_n, 1)) / discounts.reshape((1, _n))

    lambda_mtr = - matr_elem_DCG - matr_elem_DCG.T + matr_swap_elem_DCG + matr_swap_elem_DCG.T
    no_null_swap = ((order <= k).reshape((_n, 1)) + (order <= k).reshape((1, _n))) > 0
    lambda_mtr = np.abs(lambda_mtr * no_null_swap)
    if max_DCG != 0:
        return lambda_mtr / max_DCG
    else:
        return lambda_mtr

def AverageNDCG(data, y_score, k):
    uniq_queries = np.unique(data[:, -1])
    ndcg = 0
    for q in uniq_queries:
        filt_q = (data[:, -1] == q)
        ndcg += NDCGScore(data[filt_q, 0], y_score[filt_q], k)
    return ndcg / uniq_queries.shape[0]

def CountErrorPair(data, y_score):
    uniq_queries = np.unique(data[:, -1])
    count_pair = 0
    for q in uniq_queries:
        filt_q = (data[:, -1] == q)
        y_true = data[filt_q, 0]
        right_order = np.argsort(y_true)[::-1]
        y_true = y_true[right_order]
        y_pred = y_score[filt_q][right_order]

        _n = y_pred.shape[0]
        order = np.empty(y_pred.shape)
        sort_y_pred = sorted(zip(y_pred, range(_n)), key=lambda x: x[0], reverse=True)
        order[list(map(lambda x: x[1], sort_y_pred))] = range(1, _n + 1)
        pairs = ((order.reshape((_n, 1)) - order.reshape((1, _n))) < 0)
        no_null = ((y_true.reshape((_n, 1)) - y_true.reshape((1, _n))) != 0)
        count_pair += sum(sum((np.tril(pairs * no_null))))

    return count_pair

def relNormalize2(rel):
    uniq_rel = np.unique(rel)
    uniq_rel = sorted(uniq_rel)
    norm_rel = np.empty(rel.shape)
    for i, val in enumerate(uniq_rel):
        norm_rel[rel==val] = i + 1
    return norm_rel * 5


class LambdaRankForest:
    def __init__(self, learning_rate, n_estimators, sigma, start_depth):
        self._learning_rate = learning_rate
        self._n_estimators = n_estimators
        self._sigma = sigma
        self._start_depth = start_depth
        self._trees = []

    def fit(self, DATA, persent_valid, batch_size, normalize, T_NDCG):
        all_queries = DATA[:, -1]
        uniq_queries = np.unique(all_queries)
        for q in uniq_queries:
            filt = (all_queries == q)
            DATA[filt, 0] = normalize(DATA[filt, 0])
            order = np.argsort(DATA[filt, 0])[::-1]
            DATA[filt] = DATA[filt][order]

        random.shuffle(uniq_queries)
        count_valid = int(persent_valid * uniq_queries.shape[0])

        valid_queries = uniq_queries[:count_valid]
        train_queries = uniq_queries[count_valid:]
        DATA_valid = np.concatenate([DATA[all_queries == q] for q in valid_queries], axis=0)
        DATA_train = np.concatenate([DATA[all_queries == q] for q in train_queries], axis=0)
        v_queries = DATA_valid[:, -1]
        t_queries = DATA_train[:, -1]

        self._trees = []
        #tree = DT()
        #tree.fit(DATA_train[:, 1:-1], DATA_train[:, 0])
        #self._trees.append(tree)
        #h_train = tree.predict(DATA_train[:, 1:-1])
        #h_valid = tree.predict(DATA_valid[:, 1:-1])
        h_train = np.zeros(DATA_train.shape[0])#np.array([random.gauss(mu=0, sigma=0.1) for i in range(DATA_train.shape[0])])
        h_valid = np.zeros(DATA_valid.shape[0])

        iteration = 0
        while True:            
            random.shuffle(train_queries)
            for i in range(int(train_queries.shape[0] / batch_size)):
                batch_queries = train_queries[i * batch_size: (i + 1) * batch_size]
                data = np.concatenate([DATA_train[t_queries == q] for q in batch_queries], axis=0)
                h_pred = np.concatenate([h_train[t_queries == q] for q in batch_queries])
                X = data[:, 1:-1]
                Y_true = data[:, 0]

                grad = np.zeros(Y_true.shape)
                for q in batch_queries:
                    filter_query = (data[:, -1] == q)
                    h = h_pred[filter_query]
                    y = Y_true[filter_query]
                    p = 1 + np.exp((-h.reshape((h.shape[0], 1)) + h.reshape((1, h.shape[0]))) * self._sigma)
                    g = (- self._sigma / p) #* deltaNDCGScore(y, h, T_NDCG)
                    filt = ((y.reshape((y.shape[0], 1)) - y.reshape((1, y.shape[0]))) != 0)
                    g *= filt

                    g = np.tril(g)
                    xx = np.sum(g, axis=0) - np.sum(g.T, axis=0)
                    grad[filter_query] = np.sum(g, axis=0) - np.sum(g.T, axis=0)

                new_tree = DT(max_depth=self._start_depth)
                new_tree.fit(X, -grad)
                self._trees.append(new_tree)

                h_train += self._learning_rate * new_tree.predict(DATA_train[:, 1:-1])
                h_valid += self._learning_rate * new_tree.predict(DATA_valid[:, 1:-1])

                print(iteration,
                     CountErrorPair(DATA_train, h_train),
                     CountErrorPair(DATA_valid, h_valid),

                     AverageNDCG(DATA_train, h_train, 20),
                     AverageNDCG(DATA_valid, h_valid, 20),
                     AverageNDCG(DATA_train, h_train, 5),
                     AverageNDCG(DATA_valid, h_valid, 5),
                     np.linalg.norm(-grad),
                     min(-grad),
                     max(-grad))
                 #if (iteration == self._n_estimators):
                 #  break
            iteration += 1
            if (iteration == self._n_estimators):
                break

    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        for tree in self._trees:
            y_pred += self._learning_rate * tree.predict(X)
        return y_pred

def loadData():
    trainPath = "../data/train.data.cvs"
    return DataFrame.from_csv(trainPath, index_col=False).as_matrix()



In [2]:
rowData = loadData()
estimator = LambdaRankForest(learning_rate=0.01, n_estimators=100, sigma=1, start_depth=5)
estimator.fit(DATA=rowData, persent_valid=0.2, batch_size=100, normalize=relNormalize2, T_NDCG=30)


0 7113895 1701668 0.275989141943 0.262630197077 0.235925855003 0.217294238686 4791.84051279 -124.5 213.5
0 8468706 2050739 0.295417163462 0.281304375061 0.240832512657 0.226386152938 4939.89750865 -145.070426875 153.638530511
0 9084164 2202678 0.314644622287 0.308923431025 0.253681096515 0.249207837348 5153.16135799 -131.557803398 221.865987354
0 9546967 2298959 0.324228535981 0.316900277462 0.2556109273 0.245615378807 4665.94588759 -194.513014767 199.593105806
0 9639643 2323256 0.324875444881 0.321655526065 0.255085587945 0.249755214596 4592.78645582 -208.263352352 270.297917544
0 9680400 2338475 0.326349353803 0.319212490157 0.25721175175 0.248328634593 5082.42001273 -151.246130315 244.74663774
0 9809606 2358543 0.324248977635 0.317215006331 0.254774035272 0.244909049474 4121.25107766 -140.659394504 258.152651073
0 9838788 2356440 0.325973644316 0.320076387946 0.253906975239 0.246730958002 4338.08625964 -160.006997133 232.958191986
0 9894528 2369550 0.32321088169 0.316284553635 0.250

KeyboardInterrupt: 

In [1]:
def saveResults(queries, rels, namefile):
    uniq_queries = np.unique(queries)
    ans = np.empty((rels.shape[0], 2), dtype=np.int)
    count_last = 0
    for q in uniq_queries:
        rel = rels[queries == q]
        order = np.argsort(rel)[::-1] + 1 + count_last
        ans[queries == q, 0] = order 
        ans[queries == q, 1] = q
        count_last += q.shape[0]
    df = DataFrame(ans, columns=["DocumentId","QueryId"])
    df.to_csv(open(namefile, "w"), index=False)        

In [2]:
testRow  = DataFrame.from_csv("../data/testset.cvs", index_col=False).as_matrix()
ans = estimator.predict()
saveResults(testRow[:, -1], ans, "result4")

NameError: name 'DataFrame' is not defined